# I. Tweet Data Cleaning



In [1]:
## Tweet Table

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time
import re
import datetime
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', -1)

In [3]:
# function to cheack whether an object (data) contains attribute (x) or not
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [4]:
%%time
# making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input("Do you want to drop collection (yes/no): ")
if(want_drop == 'yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

Do you want to drop collection (yes/no): no
CPU times: user 5.3 ms, sys: 3.65 ms, total: 8.95 ms
Wall time: 2.61 s


In [5]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [6]:
# yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
# Kanya 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-3'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-3'
file = '/Users/huiwang/Downloads/corona-out-3'

In [7]:
%%time
# id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en') :
            
            # case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at'])
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                # check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                # check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               
 
            # case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at'])
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                # check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                # check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            # if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at'])
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                # check whether there is extension version of that tweet or not  
                if (check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            # if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            # if it is not, we insert it    
            else :
                tweet.insert_one(tw)

CPU times: user 1min 37s, sys: 5.99 s, total: 1min 43s
Wall time: 29min 47s


'id_1'

In [7]:
# create indexes for tweet id, text, hashtags and created_at
tweet.create_index([ ('id', 1) ])
tweet.create_index([ ('text', 1) ])
tweet.create_index([ ('hashtags', 1) ])
tweet.create_index([ ('created_at', 1) ])

'created_at_1'

In [8]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table

_id                   id             user_id  \
0      5ea8cd2e82a63f13f9690b8a  1253949413191344128  207809313            
1      5ea8cd2e82a63f13f9690b8b  1253992905703862272  858859031464751104   
...                         ...                  ...                 ...   
19074  5ea8d42982a63f13f969560c  1254059747738816512  3130596629           
19075  5ea8d42982a63f13f969560d  1254059751379361793  2279976427           

               created_at  \
0     2020-04-25 07:30:12   
1     2020-04-25 10:23:01   
...                   ...   
19074 2020-04-25 14:48:37   
19075 2020-04-25 14:48:38   

                                                                                                                                                                                                                                                                                                   text  \
0      India’s war with Corona is ongoing.\n\nPlay your part and make sure no one around you starves.\n\nYour contribution to defeat hunger is as valuable as defeating Corona.\n\n#FeedTheNeedy https://t.co/TUWf2nqPgZ                                                                                  
1      VIDEO | 25.04.20\n\n&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona... Turning on the Music industry.\n\n🔗https://t.co/kW5hsP5DeD\n\n#몬스타엑스 #MONSTAX\n#판타지아엑스 #FANTASIA_X\n@OfficialMonstaX\n\nhttps://t.co/tJN4Iyv1Gh https://t.co/GukJXWjaPW                              
...                                                                                                                                                                                                                                                                      ...                              
19074  We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump  #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp                                                                                                                                                     
19075  @VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with  corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona   

       retweet_count  favorite_count  reply_count  \
0      398            2144            108           
1      59             144             1             
...    ..             ...            ..             
19074  0              0               0             
19075  0              0               0             

                                   hashtags  
0      [FeedTheNeedy]                        
1      [몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]  
...                                     ...  
19074  [Corona, ashamed]                     
19075  []                                    

[19076 rows x 9 columns]





# II. User Data Cleaning





In [9]:
## User Table

In [10]:
## Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

NameError: name 'psycopg2' is not defined

In [11]:
import pandas as pd
import psycopg2
import json

In [12]:
# Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
#cur.execute("drop table users_data")
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id bigint PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [13]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [28]:
%%time
# This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET name=EXCLUDED.name, user_name=EXCLUDED.user_name, verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            # insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

NameError: name 'file' is not defined

In [14]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [15]:
user_table = create_pandas_table("SELECT * FROM users_data")
user_table

user_id        name    user_name  verified_status  \
0      375777294            TeéLaneeë🌺  TWD40_       False             
1      1132273796138905600  Terri Kamp  RampTheresa  False             
...                    ...         ...          ...    ...             
15671  734263483173212160   dirty d 😽💙  deannarae21  False             
15672  2279976427           Venugopi    Venu7630     False             

       followers_count  friends_count  statuses_count user_location  \
0      637              408            18976           None           
1      1522             256            20861           None           
...     ...             ...              ...            ...           
15671  684              482            43705           baerobbie 💙    
15672  3                35             42              None           

       favourites_count  
0      1325              
1      29166             
...      ...             
15671  75847             
15672  83                

[15673 rows x 9 columns]





# III. Searching Application





Implement 1. Top 3 Popular Original Tweet

In [23]:
%%time
# using MongoDB
# Top 3 popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_tweet = pd.DataFrame(tweet.find({}, {'user_id': 1, 'created_at': 1, 'text': 1, 'retweet_count': 1, 'favorite_count': 1, 'reply_count': 1, 'hashtags': 1}).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)]).limit(10))
pop_tweet.head(3)

CPU times: user 3.51 ms, sys: 1.82 ms, total: 5.33 ms
Wall time: 73.5 ms


_id              user_id          created_at  \
0  5ea8cf6a82a63f13f9692cfe  1100261477989126145 2020-03-13 00:43:40   
1  5ea8ced182a63f13f969264d  1112592502727548928 2020-03-18 17:51:18   
2  5ea8cde182a63f13f9691acb  1131227186          2020-03-10 17:52:14   

                                                                                                                                        text  \
0  *corona virus enters my body*\n\nThe 4 Flintstone gummies I ate in 2005: https://t.co/3STfdIQtaT                                            
1  When this Corona shit passes we have to promise each other that we’re going to tell our kids that we survived a zombie apocalypse in 2020   
2  THIS MAN IS A GENIUS he figured out the Corona virus problem 😮 https://t.co/EZP7IqTtxV                                                      

   retweet_count  favorite_count  reply_count hashtags  
0  237307         811062          1811         []       
1  181584         764405          1595         []       
2  179037         515867          1578         []

Implement 2. Top 10 Popular Hashtag

In [24]:
# using MongoDB
# TOP 10 popular hashtag ranked by aggregating the lists of hashtags (case insensitive)
from bson.son import SON
pipeline = [
     {"$unwind": "$hashtags"},
     {"$group": {"_id": {"$toLower": "$hashtags"}, "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
] 

import pprint
pprint.pprint(list(db.tweet.aggregate(pipeline))[0:10])

[{'_id': 'corona', 'count': 1248},
 {'_id': 'covid19', 'count': 542},
 {'_id': 'coronavirus', 'count': 384},
 {'_id': 'covid_19', 'count': 292},
 {'_id': 'lockdown', 'count': 166},
 {'_id': 'covid', 'count': 129},
 {'_id': 'indiafightscorona', 'count': 119},
 {'_id': 'संतरामपालजी_का_सत्संग_सुनें', 'count': 114},
 {'_id': 'stayhome', 'count': 102},
 {'_id': 'covidー19', 'count': 72}]


Implement 3. Top Active Verified User & Top Popular Verified User

In [25]:
# using SQL
# Top 3 active verified users ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY statuses_count DESC LIMIT 10")
active_user.head(3)

user_id             name       user_name  verified_status  \
0  47596019   Liputan6.com     liputan6dotcom  True              
1  16973333   The Independent  Independent     True              
2  378809160  𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ      Gidi_Traffic    True              

   followers_count  friends_count  statuses_count      user_location  \
0  3687560          693            1381273         Jakarta Indonesia   
1  3202840          1146           976895          London, England     
2  1676895          5826           922265          Everywhere          

   favourites_count  
0  7062              
1  3                 
2  14478

In [30]:
# Top 3 popular verified users ranked by followers_count
pop_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY followers_count DESC LIMIT 10")
pop_user.head(3)

user_id                name     user_name  verified_status  \
0  18839785   Narendra Modi       narendramodi  True              
1  807095     The New York Times  nytimes       True              
2  145125358  Amitabh Bachchan    SrBachchan    True              

   followers_count  friends_count  statuses_count  user_location  \
0  55786179         2364           26559           India           
1  46361159         904            393823          New York City   
2  41596464         1833           66428           Mumbai, India   

   favourites_count  
0  0                 
1  18483             
2  75

Implement 4. Top Active Location

In [27]:
# using SQL
# Top 3 active location ranked by tweets count
pop_location = create_pandas_table("SELECT user_location, count(*) as amount FROM users_data WHERE user_location != 'None' GROUP BY user_location ORDER BY amount DESC LIMIT 10")
pop_location.head(3)

user_location  amount
0  India             363   
1  New Delhi, India  185   
2  United States     125





Way to join the tweet table and user table

In [28]:
%%time
# using Pandas
full_table = pd.merge(tweet_table, user_table, how = 'left', on = ['user_id'])
full_table

CPU times: user 47.8 ms, sys: 68.3 ms, total: 116 ms
Wall time: 123 ms


_id                   id             user_id  \
0      5ea8cd2e82a63f13f9690b8a  1253949413191344128  207809313            
1      5ea8cd2e82a63f13f9690b8b  1253992905703862272  858859031464751104   
...                         ...                  ...                 ...   
19074  5ea8d42982a63f13f969560c  1254059747738816512  3130596629           
19075  5ea8d42982a63f13f969560d  1254059751379361793  2279976427           

               created_at  \
0     2020-04-25 07:30:12   
1     2020-04-25 10:23:01   
...                   ...   
19074 2020-04-25 14:48:37   
19075 2020-04-25 14:48:38   

                                                                                                                                                                                                                                                                                                   text  \
0      India’s war with Corona is ongoing.\n\nPlay your part and make sure no one around you starves.\n\nYour contribution to defeat hunger is as valuable as defeating Corona.\n\n#FeedTheNeedy https://t.co/TUWf2nqPgZ                                                                                  
1      VIDEO | 25.04.20\n\n&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona... Turning on the Music industry.\n\n🔗https://t.co/kW5hsP5DeD\n\n#몬스타엑스 #MONSTAX\n#판타지아엑스 #FANTASIA_X\n@OfficialMonstaX\n\nhttps://t.co/tJN4Iyv1Gh https://t.co/GukJXWjaPW                              
...                                                                                                                                                                                                                                                                      ...                              
19074  We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump  #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp                                                                                                                                                     
19075  @VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with  corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona   

       retweet_count  favorite_count  reply_count  \
0      398            2144            108           
1      59             144             1             
...    ..             ...            ..             
19074  0              0               0             
19075  0              0               0             

                                   hashtags             name      user_name  \
0      [FeedTheNeedy]                        BJP              BJP4India       
1      [몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]  Monsta X Europe  MonstaXEurope   
...                                     ...              ...            ...   
19074  [Corona, ashamed]                     Arkansas Boy     ArkansasWatch   
19075  []                                    Venugopi         Venu7630        

       verified_status  followers_count  friends_count  statuses_count  \
0      True             13483759         3              200328           
1      False            27084            463            38058            
...      ...              ...            ...              ...            
19074  False            276              867            4411             
19075  False            3                35             42               

                        user_location  favourites_count  
0      6-A, Deen Dayal Upadhyay Marg,  0                 
1      let the haters bark             32020             
...                    ...               ...             
19074  #Ark #LittleRock                1128              
19075  None                            83                

[19076 rows x

Implement 5. Top Earliest Original Tweet

In [34]:
%%time
# try to search the earliest created original tweet that is still getting retweeted in the time slot.
full_table['created_at'] = pd.to_datetime(full_table['created_at'])
full_table.sort_values(by = ['created_at'], ascending = (True)).drop(['_id','id'], axis = 1).head(3)

CPU times: user 15.6 ms, sys: 1.88 ms, total: 17.5 ms
Wall time: 17.9 ms


user_id          created_at  \
14241  1242575784 2013-06-03 15:32:48   
6794   189011442  2017-06-08 16:14:48   
15687  29280466   2020-01-25 18:30:29   

                                                                                                                                                                                                                                                                                                                                                        text  \
14241  Corona virus....its coming                                                                                                                                                                                                                                                                                                                              
6794   But to return to a moment of grace.\n\n"Take me to the Seine, let us gaze into it until we become little fishes &amp; recognize each other again." https://t.co/WfLNI92buL || "I have often reflected that 'Corona' is your most beautiful poem"\nIB, 6 June 1949\n\nI think so, too-with Celan echoing Rilke, 'es ist Zeit.' https://t.co/CS41BlJ4jw   
15687  I bet the marketing people at Corona really wish this thing was called Heinekenfluenza.                                                                                                                                                                                                                                                                 

       retweet_count  favorite_count  reply_count hashtags        name  \
14241  73776          131225          9968         []       Marco        
6794   16             39              0            []       coldplums    
15687  306            797             23           []       Rick Aaron   

           user_name  verified_status  followers_count  friends_count  \
14241  Marco_Acortes  False            5652             106             
6794   coldplums      False            4341             1435            
15687  RickAaron      False            12391            12854           

       statuses_count user_location  favourites_count  
14241  3839            None          4842              
6794   24661           on the road   41167             
15687  78740           None          138699

Implement 6. Search for Text

In [30]:
%%time
## user interactive search for text (case insensitive)
search_keyword = input()
user_search = full_table[full_table['text'].str.contains(search_keyword, case = False)].drop(['_id', 'id', 'user_id'], axis = 1)
user_search.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

cancel
CPU times: user 105 ms, sys: 6.54 ms, total: 111 ms
Wall time: 28.5 s


created_at  \
524   2020-04-25 03:31:42   
6701  2020-04-24 11:43:30   
...                   ...   
18399 2020-04-25 14:43:01   
18979 2020-04-25 14:47:54   

                                                                                                                                                                                                                                                                                                          text  \
524    Corona? cancelled https://t.co/XHpTaCNC3t                                                                                                                                                                                                                                                                 
6701   Today my beautiful daughter turns 11 🥳🧁 \nI had to cancel her birthday party due to Corona 😞\nI’m doing my best to make her birthday special regardless of the current state of events 🤘🏻\nSo, show Morgan some love in the comments and I’ll share them with her 🎉\nPls share! https://t.co/I5kUXKaJb3   
...                                                                                                                                                                                                                                                                                                        ...   
18399  When you realize corona can't \ncancel your exams https://t.co/1tbhUlIt37                                                                                                                                                                                                                                 
18979  @DIPRRajasthan Sir ,Best regards, due to Corona and Lok Down, Jaipur's company Traction on Demand canceled this year's placements, B.TEK students became unemployed, Sir please check this company and make efforts to get theplacements back. All students will be grateful to you                       

       retweet_count  favorite_count  reply_count hashtags  \
524    347            4447            122          []        
6701   137            1468            677          []        
...    ...             ...            ...          ..        
18399  0              0               0            []        
18979  0              0               0            []        

                         name        user_name  verified_status  \
524    aksel                   aksually         False             
6701   𝐒𝐡𝐚𝐧𝐧𝐨𝐧 𝐢𝐧 𝐑𝐞𝐚𝐥 𝐋𝐢𝐟𝐞 ♡  Im_shaleelee     False             
...                       ...           ...       ...             
18399  @meme_se_marunga        Memewala11       False             
18979  Roop Chand Jain         RoopCha58911462  False             

       followers_count  friends_count  statuses_count user_location  \
524    94104            605            5941            he/him         
6701   4735             1287           9729            None           
...     ...              ...            ...             ...           
18399  0                27             368             None           
18979  3                83             54              None           

       favourites_count  
524    6403              
6701   16781             
...      ...             
18399  8                 
18979  35                

[99 rows x 14 columns]

Implement 7. Search for Hashtag

In [32]:
%%time
## user interactive search for hashtag (case insensitive)

search_keyword = input()

target = [search_keyword]
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id', 'id', 'user_id'], axis = 1)
lockdown.sort_values(by = ['followers_count'], ascending = (False))

stayhome
CPU times: user 303 ms, sys: 10.6 ms, total: 313 ms
Wall time: 12.6 s


created_at  \
416   2020-04-25 10:21:38   
1092  2020-04-24 17:46:30   
...                   ...   
14217 2020-04-25 14:10:32   
17981 2020-04-25 14:40:05   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      text  \
416    People take oath of #SocialDistancing and other measures to contain corona pandemic at Bettia, Bihar.\n\n#IndiaFightsCorona #StayHome \n\n#Report Asis Kumar https://t.co/Zp7RxAhIJo                                                                                                                                                                                                                                                                                                                                                                                                  
1092   Generosity is a core American value that I'm always proud to see on display in our partnerships. During this pandemic, many families are facing challenging times. Inspired to see U.S. companies pitching in to take care of their neighbors. #partners4prosperity || U.S. companies are supporting Pakistan’s efforts against corona! \n\nU.S. company @Excelerate_LP partnered with @KarachiRelief to donate a month’s supply of food and household goods to 5,000n Karachi families in need, delivered by @CareemPAK\n\n#PakistanFightsCorona #StayHome https://t.co/oOib6DBUA4   
...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ...   
14217  Stay home🏘 Stay safe👨‍👩‍👧‍👦 #coronahelplineworld #covid19 #live #travel #help #stayhome #stayhomestaysafe #coronavirus #corona @ PLANET https://t.co/fjPFQOex3u https://t.co/3ERW5AfJ3o                                                                                                                                                                                                                                                                                                                                                                                               
17981  @narendramodi super corona fighter #DilSeThankYou #StayHome #StayHomeStaySafe #Covid_19 So lucky to have you as our prime minister. Thank you sir https://t.co/ksShRqfeh9                                                                                                                                                                                                                                                                                                                                                                                                             

       retweet_count  favorite_count  reply_count  \
416    8              50              3             
1092   495            2180            169           
...    ...             ...            ...           
14217  0              0               0             
17981  0              0               0             

                                                                            

Text, hashtag, and time search

In [16]:
import time
import datetime
import timeit

# set for storing keyword
stored_keyword = set()
stored_hashtag = set()
stored_date = set()
# dictionary of dictionaries
dict_text = {}
dict_hashtag = {}
dict_date = {}

# function: store dataframe in dictionary 
def put_into_cache(keyword, df, dict_set) :
    if (len(dict_set) == 10): # we can hold 10 caches if it over, we deleted oldest one
        del dict_set[list(dict_set.keys())[0]] 
    
    dict_set[keyword] = df.to_dict()

In [216]:
# function: read cache from dictionary to dataframe
def dict_to_df(keyword, dict_set) :
    return pd.DataFrame.from_dict(dict_set[keyword])

In [217]:
def text_search():
    user_keyword = input("Please enter a keyword: ")
    start = timeit.default_timer()

    regx = re.compile(user_keyword, re.IGNORECASE)
    
    if (user_keyword.lower() not in stored_keyword) :
        search_tweet = pd.DataFrame(
            tweet.find({'text': {"$regex": regx}}).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)])
        )

        if (len(search_tweet) == 0) :
            print("Error: No result for %s" %user_keyword)
            search_full = None

        else :
            s_user = tuple(search_tweet["user_id"].tolist())
            params = {'l': s_user}

            cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
            search_user = pd.DataFrame(cur.fetchall())
            search_user.columns = [i[0] for i in cur.description]

            search_full = pd.merge(search_tweet, search_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)

            stored_keyword.add(user_keyword.lower())
            put_into_cache(user_keyword.lower(), search_full, dict_text)

    else :
        search_full = dict_to_df(user_keyword.lower(), dict_text)    

    stop = timeit.default_timer()
    
    print('Running Time: ', "%.3f second" %(stop - start))
    return search_full

In [218]:
def hashtag_search():
    user_keyword = input("Please enter a keyword: ")
    start = timeit.default_timer()
    
    if (user_keyword.lower() not in stored_hashtag) :
        # uses the index to find
        hashtag_tweet = pd.DataFrame(
            db.tweet.find({"hashtags": {"$in": [user_keyword]}}).collation( { "locale": 'en', "strength": 2 } ).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)])
        )

        if (len(hashtag_tweet) == 0) :
            print("Error: No result for %s" %user_keyword)
            hash_search_table = None

        else :
            h_user = tuple(hashtag_tweet["user_id"].tolist())
            params = {'l': h_user}

            cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
            hash_user = pd.DataFrame(cur.fetchall())
            hash_user.columns = [i[0] for i in cur.description]

            hash_search_table = pd.merge(hashtag_tweet, hash_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)

            stored_hashtag.add(user_keyword.lower())
            put_into_cache(user_keyword.lower(), hash_search_table, dict_hashtag)

    else :
        hash_search_table = dict_to_df(user_keyword.lower(), dict_hashtag)    
    
    stop = timeit.default_timer()
    print('Running Time: ', "%.3f second" %(stop - start))
    return hash_search_table

In [231]:
def time_search():
    period = True
    while period :
        dateinput1 = input("Please enter the start date (format: yyyy-m-d (ex: 2020-4-13)): ")
        x1 = [int(x) for x in dateinput1.split("-")]
        dateinput2 = input("Please enter the end date (format: yyyy-m-d (ex: 2020-4-15)): ")
        x2 = [int(x) for x in dateinput2.split("-")]

        start = datetime.datetime(x1[0], x1[1], x1[2], 0, 0, 0, 0)
        end = datetime.datetime(x2[0], x2[1], x2[2], 23, 59, 59, 0)

        if (start <= end) :
            period = False
        else :
            print("Error! Try again. Start date should be earlier than end date.")
            
    start_time = timeit.default_timer()
    
    if ((dateinput1 + " to " + dateinput2) not in stored_date) :
    
        ##sort it by time (asc)
        range_tweet = pd.DataFrame(tweet.find({'created_at': {"$gte": start, "$lte": end}}).sort([("created_at", 1)]))

        if (len(range_tweet) == 0) :
            print("Error: No result for input range!")
            range_search_table = None
        else :
            r_user = tuple(range_tweet["user_id"].tolist())
            params = {'l': r_user}

            cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
            range_user = pd.DataFrame(cur.fetchall())
            range_user.columns = [i[0] for i in cur.description]

            range_search_table = pd.merge(range_tweet, range_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)

            stored_date.add(dateinput1 + " to " + dateinput2)
            put_into_cache(dateinput1 + " to " + dateinput2, range_search_table, dict_date)

    else :
        range_search_table = dict_to_df(dateinput1 + " to " + dateinput2, dict_date)            
    
    stop_time = timeit.default_timer()
    print('Running Time: ', "%.3f second" %(stop_time - start_time))
    return range_search_table

In [220]:
text_search()

Please enter a keyword: covid-19
Running Time:  0.106 second


created_at  \
0   2020-04-15 18:23:49   
1   2020-03-22 05:52:25   
..                  ...   
491 2020-04-25 14:47:54   
492 2020-04-25 14:48:22   

                                                                                                                                                                                                                                                                                                                                                                                                                            text  \
0    Israel is shutting down COVID-19 Clinics in PALESTINE and THEY ARRESTED THE STAFF. Palestinians don’t even have the RIGHT TO GET TESTED FOR CORONA DURING A PANDEMIC! I’m so tired of seeing my peoples BASIC HUMAN RIGHTS CONSTANTLY being violated.                                                                                                                                                                         
1    Can you see the link?\n\nLooks like people have closed ranks to uproot the COVID-19 menace. #JantaCurfew || Have seen this view many times from my hotel room over the years but not with less than 1000 cars on it........#India is having a curfew today for 14 hours to fight #covid_19 it’s looks like it’s being followed 🙏🙏🙏\n.\n.\n#jantacurfew #modi #stayathome #corona #covid_19 #seagate https://t.co/KsY5adOjQX   
..                                                                                                                                                                                                                                                                                                                                                                                                                           ...   
491  Let's pray for the world nd people affected by new corona virus (covid-19) 🙏😔\n#stayathome \n#staysafe https://t.co/iXke2MY2iK                                                                                                                                                                                                                                                                                                
492  @Zaynab_bin @AjitKDoval_FAN @fawadchaudhry You idiot, the best doctors are saying HCQ is not a cure for Covid-19, just fraud.  #Pakistan doesn't want HCQ; China is sending the vaccine here.  Pretty soon, you will be crying that China didn't send it to you.  Corona is yet to reach its peak in #Bharat.                                                                                                                 

     retweet_count  favorite_count  reply_count  \
0    27394          38302           95            
1    6346           35558           789           
..    ...             ...           ...           
491  0              0               0             
492  0              0               0             

                                                                                     hashtags  \
0    []                                                                                         
1    [JantaCurfew, India, covid_19, jantacurfew, modi, stayathome, corona, covid_19, seagate]   
..                                                                                        ...   
491  [stayathome, staysafe]                                                                     
492  []                                                                                         

                      name       user_name  verified_status  followers_count  \
0    Girl 1n Spacê          imagirlinspace  False            1419              
1    Narendra Modi          narendramodi    True             55786179          
..             ...                   ...     ...                  ...          
491  Hridayandra_Jung_Shah  HridayandraT    False            1062              
492  Aamir Yousafi          AamirYousaf

In [221]:
hashtag_search()

Please enter a keyword: COVId
Running Time:  0.070 second


created_at  \
0   2020-04-22 08:50:04   
1   2020-04-19 19:13:32   
..                  ...   
124 2020-04-25 14:47:02   
125 2020-04-25 14:47:49   

                                                                                                                                                                                                                                                                                                                                                                                                                                    text  \
0    To all JSP leaders and Janasainiks please spread the message of ‘ Sri Moulana Moheebullah - Imam of Jama Masjid’ that all Indians who are practising ‘Islam religion‘; kindly, to follow his guidelines in this Holy month of Ramzaan || Moulana Moheebullah - Imam of Jama Masjid requesting all Muslims to stay at home and pray. Let’s fight Corona in the best way possible 🙏. \n#Ramzaan #Covid #IMPAR https://t.co/YvTEv6WKsO   
1    Chinese police assaulting a black man. They are all racists. China must know that no nation is indispensable. \nKindly #Rt and follow my handle. \n#Corona #simi #COVID #slimcase #اجلاء_الكويتيين #COVIDー19 #ChinaMustPay #ChinaLiedPeopleDied #ChinaVirus #Chinazi #chinaracism #cpp https://t.co/AKXbsUkvSG                                                                                                                        
..                                                                                                                                                                                                                                                                                                              ...                                                                                                                        
124  Dancing the Corona Blues Away by @marilynflower27 https://t.co/CW9uMuQshC #Poetry #COVID19 #Dance #Self #SelfLove #COVID #Coronavirus                                                                                                                                                                                                                                                                                                 
125  Nice efforts guys..keep up the gud work✌\n#EIAForGreenIndia #covid\n#lockdown\n#Corona || This is a mechanism to legitimise some actions currently listed as violations,like projects starting construction without a valid clearance, dilutes rules by expanding the list of projects exempted from public consultation, it does not prescribe a robust system.\n#EIAForGreenIndia https://t.co/DmaY23TTvO                           

     retweet_count  favorite_count  reply_count  \
0    2435           7164            65            
1    1193           1412            251           
..    ...            ...            ...           
124  0              0               0             
125  0              0               0             

                                                                                                                                     hashtags  \
0    [Ramzaan, Covid, IMPAR]                                                                                                                    
1    [Rt, Corona, simi, COVID, slimcase, اجلاء_الكويتيين, COVIDー19, ChinaMustPay, ChinaLiedPeopleDied, ChinaVirus, Chinazi, chinaracism, cpp]   
..                                                                                                                                        ...   
124  [Poetry, COVID19, Dance, Self, SelfLove, COVID, Coronavirus]                                                                               
125  [EIAForGreenIndia, covid, lockdown, Corona, EIAForGreenIndia]                                                                              

                      name        user_name  verified_status  followers_count  \
0    Pawan Kalyan     

In [232]:
time_search()

Please enter the start date (format: yyyy-m-d (ex: 2020-4-13)): 2020-4-11
Please enter the end date (format: yyyy-m-d (ex: 2020-4-15)): 2020-4-30
Running Time:  1.138 second


created_at  \
0     2020-04-11 02:48:37   
1     2020-04-11 03:04:17   
...                   ...   
18703 2020-04-25 14:48:37   
18704 2020-04-25 14:48:38   

                                                                                                                                                                                                                                                                                                          text  \
0      I convey my greetings to all Christian Brothers and Sisters on Good Friday. May Lord bless us and give us  strength to fight Corona Virus unitedly as One.                                                                                                                                                
1      Happy Morning\nTo all nationalist brothers and sisters . this morning will be give you new energy.\nWe will be victorious by breaking all the intrigues in the battle of corona virus.\nश्री राम\n#GodMorningSaturday\n#SaturdayThoughts\n#SaturdayMotivation\n#SaturdayMorning https://t.co/GrVFzvu9HF   
...                                                                                                                                                                                                                                                                                                        ...   
18703  We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump  #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp                                                                                                                                                            
18704  @VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with  corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona          

       retweet_count  favorite_count  reply_count  \
0      47             313             72            
1      65             143             18            
...    ..             ...             ..            
18703  0              0               0             
18704  0              0               0             

                                                                          hashtags  \
0      []                                                                            
1      [GodMorningSaturday, SaturdayThoughts, SaturdayMotivation, SaturdayMorning]   
...                                                                            ...   
18703  [Corona, ashamed]                                                             
18704  []                                                                            

                     name      user_name  verified_status  followers_count  \
0      digvijaya singh     digvijaya_28   True             975311            
1      🇮🇳shreya surve🇮🇳🇮🇳  shreyasurve2   False            669               
...                   ...           ...     ...            ...               
18703  Arkansas Boy        ArkansasWatch  False            276               
18704  Venugopi            Venu7630       False            3                 

       friends_count  statuses_count     user_location  favourites_count  
0      93             25882           Delhi             504               
1      75             8373            india             2625              
...    ..              ...              ...              ...              
18703  867            4411            #Ark #LittleRock  1128              
18704  35             42              None              83                

[18705 rows x 14 columns]